<a href="https://colab.research.google.com/github/4graf/Data_Processing/blob/main/ML/%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B2%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
def P(a, b, c, x1, x2):
    psi = c + a*x1 + b*x2
    return 1/(1+np.exp(-psi))

print(P(1.22, -0.12, -6.23, 2, 4))
print(P(-0.95, -0.97, 7.52, 2, 4))
print(P(-0.57, 1.09, -5.53, 2, 4))


0.01378898850614155
0.8506870654691562
0.09029814470291989


# SKLEARN

In [ ]:
# !pip install --upgrade pip
# !pip install imutils
# !pip install opencv-python
# !pip install --upgrade scikit-learn==0.23.0

! pip uninstall scikit-learn -y
! pip install scikit-learn==0.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [ ]:
! unzip "train_task (1).zip"
! unzip "test_task (1).zip"

Archive:  train_task (1).zip
   creating: train/
  inflating: train/.DS_Store         
  inflating: train/cat.0.jpg         
  inflating: train/cat.1.jpg         
  inflating: train/cat.10.jpg        
  inflating: train/cat.100.jpg       
  inflating: train/cat.101.jpg       
  inflating: train/cat.102.jpg       
  inflating: train/cat.103.jpg       
  inflating: train/cat.104.jpg       
  inflating: train/cat.105.jpg       
  inflating: train/cat.106.jpg       
  inflating: train/cat.107.jpg       
  inflating: train/cat.108.jpg       
  inflating: train/cat.109.jpg       
  inflating: train/cat.11.jpg        
  inflating: train/cat.110.jpg       
  inflating: train/cat.111.jpg       
  inflating: train/cat.112.jpg       
  inflating: train/cat.113.jpg       
  inflating: train/cat.114.jpg       
  inflating: train/cat.115.jpg       
  inflating: train/cat.116.jpg       
  inflating: train/cat.117.jpg       
  inflating: train/cat.118.jpg       
  inflating: train/cat.119.jpg       
 

In [ ]:
import cv2

def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
from os import listdir

imgs_train = sorted(['train/' + file for file in listdir('train')])[1:]
imgs_test = sorted(['test/' + file for file in listdir('test')])[2:]

In [ ]:
df_train = pd.DataFrame({'Image': imgs_train})
df_train['Animal'] = np.where(df_train['Image'].str.contains('cat'), 1, 0)
df_test = pd.DataFrame({'Image': imgs_test})
df_test['Animal'] = np.where(df_test['Image'].str.contains('cat'), 1, 0)

In [ ]:
X_train = list(map(lambda x: extract_histogram(cv2.imread(x)), df_train['Image']))
y_train = df_train['Animal']
X_test =  list(map(lambda x: extract_histogram(cv2.imread(x)), df_test['Image']))
y_test = df_test['Animal']

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [ ]:
bagging_estimator = BaggingClassifier(DecisionTreeClassifier(criterion='entropy',
                                                             min_samples_leaf=10,
                                                             max_leaf_nodes=20,
                                                             random_state=220),
                                      n_estimators=19, random_state=220)
estimators = [
    ('svr', LinearSVC(C=1.74, random_state=220)),
    ('bg', bagging_estimator),
    ('rf', RandomForestClassifier(n_estimators=19, criterion='entropy', 
                                  min_samples_leaf=10, max_leaf_nodes=20, 
                                  random_state=220))
]
clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(solver='lbfgs', random_state=220),
    cv=2
)

clf.fit(X_train, y_train)

StackingClassifier(cv=2,
                   estimators=[('svr', LinearSVC(C=1.74, random_state=220)),
                               ('bg',
                                BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                        max_leaf_nodes=20,
                                                                                        min_samples_leaf=10,
                                                                                        random_state=220),
                                                  n_estimators=19,
                                                  random_state=220)),
                               ('rf',
                                RandomForestClassifier(criterion='entropy',
                                                       max_leaf_nodes=20,
                                                       min_samples_leaf=10,
                           

In [ ]:
clf.score(X_train, y_train)

0.851

In [ ]:
y_pred = clf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[33 17]
 [19 31]]
              precision    recall  f1-score   support

           0       0.63      0.66      0.65        50
           1       0.65      0.62      0.63        50

    accuracy                           0.64       100
   macro avg       0.64      0.64      0.64       100
weighted avg       0.64      0.64      0.64       100



In [ ]:
(33+31)/100

0.64

In [ ]:
clf.predict_proba(X_test)[28]

array([0.51493032, 0.48506968])

In [ ]:
df_test.iloc[28]

Image     test/cat.1028.jpg
Animal                    1
Name: 28, dtype: object